In [1]:
import pandas as pd
import numpy as np

import json
import ast

In [2]:
book_path = 'movies_metadata'
fix_book_path = 'movies_metadata_fixed' 

In [3]:
with open(f'{book_path}.csv', 'r', encoding='utf-8') as f, open(f'{fix_book_path}.csv', 'w', encoding='utf-8') as g:
    content = f.read().replace('\n ','\t ')
    g.write(content)
    print("Fixed file {0}.csv".format(fix_book_path))

Fixed file movies_metadata_fixed.csv


## Movies & Genres

In [16]:
book   = pd.read_csv(f'{fix_book_path}.csv', low_memory=False)

In [17]:
print(book.info(verbose=True))
book

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  bool   
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  int64  
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int64  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45463 non-null  float64
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45458,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1
45459,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3
45460,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6
45461,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0


In [18]:
# book['overview'] = book['overview'].str.replace('\t ','\n ')

In [19]:
book = book.drop_duplicates(subset=['id'])
book.shape

(45433, 24)

### Genres

In [40]:
testing = book[['genres','id']].copy()

In [41]:
import json
testing['genres'] = testing['genres'].str.replace("'", '"').apply(json.loads)

In [348]:
s = testing.explode('genres')
df_json = s.to_json(orient='records')
finaldf = pd.json_normalize(json.loads(df_json), meta=['id'])

print(finaldf.info(verbose=True))
finaldf

In [349]:
finaldf = finaldf.drop('genres', axis=1).dropna()
finaldf.columns = ['movie_id', 'genre_id', 'genre_name']
finaldf['genre_id'] = finaldf['genre_id'].astype(int)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91015 entries, 0 to 93454
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   movie_id    91015 non-null  int64 
 1   genre_id    91015 non-null  int32 
 2   genre_name  91015 non-null  object
dtypes: int32(1), int64(1), object(1)
memory usage: 2.4+ MB
None


,movie_id,genre_id,genre_name
0,862,16,Animation
1,862,35,Comedy
2,862,10751,Family
3,8844,12,Adventure
4,8844,14,Fantasy
...,...,...,...
93450,439050,10751,Family
93451,111109,18,Drama
93452,67758,28,Action
93453,67758,18,Drama


In [350]:
genre_table = finaldf.groupby(['genre_id','genre_name']).size().reset_index().rename(columns = {0: 'count'})
genre_table

,genre_id,genre_name,count
0,12,Adventure,3490
1,14,Fantasy,2309
2,16,Animation,1931
3,18,Drama,20244
4,27,Horror,4671
5,28,Action,6592
6,35,Comedy,13176
7,36,History,1398
8,37,Western,1042
9,53,Thriller,7619


In [351]:
genre_table = genre_table.drop('count', axis=1)

In [352]:
filepath = "./proj/Genre"
genre_table.to_csv(f'{filepath}.csv', index=False)

In [353]:
join_table = finaldf.drop('genre_name', axis=1)

In [354]:
filepath = "./proj/join_movie_genre"
join_table.to_csv(f'{filepath}.csv', index=False)

## Movies

In [20]:
movie_only = book[['adult','id','title', 'overview', 'runtime','release_date',\
                   'vote_average', 'original_language', 'original_title', 'popularity']].copy()

In [21]:
movie_only.columns = ['adult','movie_id','title','description','duration','release_date','vote','o_lan', 'o_tit', 'pop']

In [22]:
movie_only['duration'] = movie_only['duration'].fillna(value=0).astype(int).astype(str) + ' minutes'

In [23]:
def adultcheck(x):
    if x:
        return 'C18'
    else:
        return 'P'

movie_only['age_restriction'] = movie_only['adult'].apply(adultcheck)

In [24]:
movie_only

,adult,movie_id,title,description,duration,release_date,vote,o_lan,o_tit,pop,age_restriction
0,False,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",81 minutes,1995-10-30,7.7,en,Toy Story,21.946943,P
1,False,8844,Jumanji,When siblings Judy and Peter discover an encha...,104 minutes,1995-12-15,6.9,en,Jumanji,17.015539,P
2,False,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,101 minutes,1995-12-22,6.5,en,Grumpier Old Men,11.712900,P
3,False,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",127 minutes,1995-12-22,6.1,en,Waiting to Exhale,3.859495,P
4,False,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,106 minutes,1995-02-10,5.7,en,Father of the Bride Part II,8.387519,P
...,...,...,...,...,...,...,...,...,...,...,...
45458,False,439050,Subdue,Rising and falling between a man and woman.,90 minutes,NaN,4.0,fa,رگ خواب,0.072051,P
45459,False,111109,Century of Birthing,An artist struggles to finish his work while a...,360 minutes,2011-11-17,9.0,tl,Siglo ng Pagluluwal,0.178241,P
45460,False,67758,Betrayal,"When one of her hits goes wrong, a professiona...",90 minutes,2003-08-01,3.8,en,Betrayal,0.903007,P
45461,False,227506,Satan Triumphant,"In a small town live two brothers, one a minis...",87 minutes,1917-10-21,0.0,en,Satana likuyushchiy,0.003503,P


In [360]:
movie_table = movie_only.drop(['adult', 'vote', 'o_lan', 'o_tit', 'pop'], axis=1)
movie_table = movie_table[['movie_id','title','description','age_restriction','duration', 'release_date']]

In [361]:
movie_table

,movie_id,title,description,age_restriction,duration,release_date
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",P,81 minutes,1995-10-30
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,P,104 minutes,1995-12-15
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,P,101 minutes,1995-12-22
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",P,127 minutes,1995-12-22
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,P,106 minutes,1995-02-10
...,...,...,...,...,...,...
45458,439050,Subdue,Rising and falling between a man and woman.,P,90 minutes,NaN
45459,111109,Century of Birthing,An artist struggles to finish his work while a...,P,360 minutes,2011-11-17
45460,67758,Betrayal,"When one of her hits goes wrong, a professiona...",P,90 minutes,2003-08-01
45461,227506,Satan Triumphant,"In a small town live two brothers, one a minis...",P,87 minutes,1917-10-21


In [362]:
filepath = "./proj/Movie"
movie_table.to_csv(f'{filepath}.csv', index=False)

In [368]:
movie_retest = movie_table

In [369]:
movie_retest['len'] = movie_retest['title'].apply(len)

In [370]:
movie_retest.sort_values(by='len')

,movie_id,title,description,age_restriction,duration,release_date,len
16032,49207,R,The R of the title stands for the young protag...,P,99 minutes,2010-04-22,1
33862,263471,I,A bodybuilder ends up falling madly in love wi...,P,170 minutes,2015-01-04,1
26014,53459,F,A group of teachers must defend themselves fro...,P,79 minutes,2010-09-07,1
16166,36373,M,"Remake of the 1931 original. In the city, some...",P,88 minutes,1951-03-01,1
22986,45514,8,8 shorts centered around 8 themes directed by ...,P,100 minutes,2008-10-23,1
...,...,...,...,...,...,...,...
30362,68816,Will Our Heroes Be Able to Find Their Friend W...,"A rich businessman is fed up with work, family...",P,130 minutes,1968-12-19,88
36823,270936,Ellen ten Damme: As I Was Wondering Where This...,Documentary about Dutch artist Ellen ten Damme...,P,66 minutes,2007-04-19,90
36854,316111,Riuscirà l'avvocato Franco Benenato a sconfigg...,NaN,P,0 minutes,1971-08-18,101
7241,10338,Those Magnificent Men in Their Flying Machines...,Star studded comedy about a early 20th century...,P,138 minutes,1965-06-03,104


## Directors & Casts

In [4]:
credit_path = 'credits'

In [5]:
credit   = pd.read_csv(f'{credit_path}.csv', low_memory=False)

In [6]:
print(credit.info(verbose=True))
credit

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [7]:
credit = credit.drop_duplicates(subset=['id'])
credit.shape

(45432, 3)

In [8]:
# Change dict string to nested lists?
testing = credit[['cast','crew','id']].copy()
testing['crew'] = testing['crew'].map(ast.literal_eval)
testing['cast'] = testing['cast'].map(ast.literal_eval)

### 1. Directors

In [10]:
s = testing[['crew', 'id']].explode('crew')
df_json = s.to_json(orient='records')
finaldf = pd.json_normalize(json.loads(df_json), meta=['id'])

print(finaldf.info(verbose=True))
finaldf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464607 entries, 0 to 464606
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 464607 non-null  int64  
 1   crew.credit_id     463836 non-null  object 
 2   crew.department    463836 non-null  object 
 3   crew.gender        463836 non-null  float64
 4   crew.id            463836 non-null  float64
 5   crew.job           463836 non-null  object 
 6   crew.name          463836 non-null  object 
 7   crew.profile_path  95001 non-null   object 
 8   crew               0 non-null       float64
dtypes: float64(3), int64(1), object(5)
memory usage: 31.9+ MB
None


,id,crew.credit_id,crew.department,crew.gender,crew.id,crew.job,crew.name,crew.profile_path,crew
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,NaN
1,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,NaN
2,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,NaN
3,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg,NaN
4,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg,NaN
...,...,...,...,...,...,...,...,...,...
464602,67758,52fe4776c3a368484e0c8399,Sound,0.0,549356.0,Original Music Composer,Richard McHugh,None,NaN
464603,67758,52fe4776c3a368484e0c839f,Camera,2.0,58818.0,Director of Photography,João Fernandes,None,NaN
464604,227506,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg,NaN
464605,227506,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None,NaN


In [41]:
onlydirector = finaldf.loc[finaldf['crew.job'] == 'Director'].copy()
onlydirector = onlydirector[['id', 'crew.name']]
onlydirector.columns = ['movie_id', 'director_name']
onlydirector = onlydirector.drop_duplicates() # There was one duplicate Jari Halonen
onlydirector['director_id'] = pd.factorize(onlydirector['director_name'])[0]
onlydirector

,movie_id,director_name,director_id
0,862,John Lasseter,0
109,8844,Joe Johnston,1
122,15602,Howard Deutch,2
126,31357,Forest Whitaker,3
141,11862,Charles Shyer,4
...,...,...,...
464584,439050,Hamid Nematollah,19738
464593,111109,Lav Diaz,12064
464599,67758,Mark L. Lester,1442
464604,227506,Yakov Protazanov,4755


In [42]:
director_table = onlydirector.groupby(['director_id','director_name']).size().reset_index(name='counts')
director_table

,director_id,director_name,counts
0,0,John Lasseter,10
1,1,Joe Johnston,10
2,2,Howard Deutch,11
3,3,Forest Whitaker,4
4,4,Charles Shyer,7
...,...,...,...
19735,19735,Ravi Udyawar,1
19736,19736,Shanra J. Kehl,1
19737,19737,Aaron Osborne,1
19738,19738,Hamid Nematollah,1


In [35]:
director_table = director_table.drop('counts', axis=1)

In [36]:
filepath = "./proj/Director"
director_table.to_csv(f'{filepath}.csv', index=False)

In [37]:
join_direct_movie = onlydirector[['movie_id','director_id']]

In [38]:
filepath = "./proj/join_movie_director"
join_direct_movie.to_csv(f'{filepath}.csv', index=False)

### 2. Crews

In [266]:
s = testing[['cast', 'id']].explode('cast')
df_json = s.to_json(orient='records')
crewdf = pd.json_normalize(json.loads(df_json), meta=['id'])

print(crewdf.info(verbose=True))
crewdf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564458 entries, 0 to 564457
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 564458 non-null  int64  
 1   cast.cast_id       562044 non-null  float64
 2   cast.character     562044 non-null  object 
 3   cast.credit_id     562044 non-null  object 
 4   cast.gender        562044 non-null  float64
 5   cast.id            562044 non-null  float64
 6   cast.name          562044 non-null  object 
 7   cast.order         562044 non-null  float64
 8   cast.profile_path  388366 non-null  object 
 9   cast               0 non-null       float64
dtypes: float64(5), int64(1), object(4)
memory usage: 43.1+ MB
None


,id,cast.cast_id,cast.character,cast.credit_id,cast.gender,cast.id,cast.name,cast.order,cast.profile_path,cast
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,NaN
1,862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,NaN
2,862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,NaN
3,862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,NaN
4,862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,NaN
...,...,...,...,...,...,...,...,...,...,...
564453,227506,3.0,,52fe4ea59251416c7515d7d9,1.0,1090923.0,Nathalie Lissenko,1.0,None,NaN
564454,227506,4.0,,52fe4ea59251416c7515d7dd,2.0,1136422.0,Pavel Pavlov,2.0,None,NaN
564455,227506,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None,NaN
564456,227506,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg,NaN


In [267]:
crewdf = crewdf[crewdf['cast.id'].notna()]
crewdf = crewdf[['id', 'cast.id', 'cast.character', 'cast.name']]
crewdf.columns = ['movie_id', 'cast_id', 'cast_character', 'cast_name']
crewdf['cast_id'] = crewdf['cast_id'].astype(int)
crewdf

,movie_id,cast_id,cast_character,cast_name
0,862,31,Woody (voice),Tom Hanks
1,862,12898,Buzz Lightyear (voice),Tim Allen
2,862,7167,Mr. Potato Head (voice),Don Rickles
3,862,12899,Slinky Dog (voice),Jim Varney
4,862,12900,Rex (voice),Wallace Shawn
...,...,...,...,...
564452,227506,544742,,Iwan Mosschuchin
564453,227506,1090923,,Nathalie Lissenko
564454,227506,1136422,,Pavel Pavlov
564455,227506,1261758,,Aleksandr Chabrov


In [270]:
cast_table = crewdf.groupby(['cast_id','cast_name']).size().reset_index(name='counts')
cast_table

,cast_id,cast_name,counts
0,1,George Lucas,20
1,2,Mark Hamill,47
2,3,Harrison Ford,57
3,4,Carrie Fisher,46
4,5,Peter Cushing,69
...,...,...,...
206167,1907919,Axel Danielsson,1
206168,1907920,Lasse Dithmar,1
206169,1907923,Pia Edlund,1
206170,1907926,Stina Elinderson,1


In [271]:
# Remove casts with non-English names
nonenglish = cast_table[cast_table.duplicated(subset='cast_id', keep='first')]
cast_table = cast_table.drop_duplicates(subset='cast_id', keep='first')

print(cast_table.shape)
pd.merge(nonenglish, cast_table, on='cast_id')

(206158, 3)


,cast_id,cast_name_x,counts_x,cast_name_y,counts_y
0,9779,Моррис Честнат,24,Morris Chestnut,2
1,23764,Эрика Элениак,14,Erika Eleniak,1
2,58646,Дэмиан Чапа,5,Damian Chapa,1
3,67212,Том Ву,10,Tom Wu,1
4,72932,松田龙平,11,Ryuhei Matsuda,4
5,74947,张雨绮,3,Kitty Zhang Yuqi,1
6,78456,Кевин «Кимбо Слайс» Фергюсон,3,Kimbo Slice,1
7,78809,Рашад Эванс,1,Rashad Evans,2
8,87637,长泽雅美,11,Masami Nagasawa,2
9,87662,高桥一生,8,Issei Takahashi,1


In [272]:
cast_table = cast_table.drop('counts', axis=1)

In [273]:
filepath = "./proj/Cast"
cast_table.to_csv(f'{filepath}.csv', index=False)

In [282]:
join_movie_cast = crewdf[['movie_id','cast_id']].drop_duplicates(['movie_id', 'cast_id'])

In [283]:
filepath = "./proj/join_movie_cast"
join_movie_cast.to_csv(f'{filepath}.csv', index=False)